In [88]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import ocr.page_detection
plt.rcParams['figure.figsize'] = (9.0, 9.0)

In [89]:
ANSWER_KEY = {0: 1, 1: 2, 2: 2, 3: 1, 4: 2, 5:1, 6:1, 7:1, 8:1, 9:1, 10:1, 11:1, 12:2, 13:3, 14:3, 15:3, 16:2, 17:1}

In [90]:
image = cv2.imread(r"C:\Users\Gautam\Desktop\OMR\thanos2.jpeg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

In [91]:
cv2.imwrite("image.jpg", image)
cv2.imwrite("gray.jpg", gray)
cv2.imwrite("blurred.jpg", blurred)
cv2.imwrite("edged.jpg", edged)

True

In [92]:
# find contours in the edge map, then initialize
# the contour that corresponds to the document

cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None
ok=False

# ensure that at least one contour was found
if len(cnts) > 0:
	# sort the contours according to their size in
	# descending order
	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
 
	# loop over the sorted contours
	for c in cnts:
		# approximate the contour
		peri = cv2.arcLength(c, True)
		approx = cv2.approxPolyDP(c,  0.02* peri, True)

     
		# if our approximated contour has four points,
		# then we can assume we have found the paper
		if len(approx) == 4:
			docCnt = approx
			contoured=cv2.drawContours(image.copy(), docCnt, -1, (0,255,0), 3)
            
			ok=True
			break


In [93]:
print(ok)

True


In [94]:
contoured=cv2.drawContours(image.copy(), cnts, -1, (0,255,0), 3)

In [95]:
cv2.imwrite("5contoured.jpg", contoured)

True

In [96]:
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))
paper=cv2.resize(paper, (430,600))
warped=cv2.resize(paper, (430,600))

## EXPERIMENTAL LINE

In [97]:
##  cnts = cv2.findContours(warped.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[0]
## cv2.drawContours(warped, cnts, -1, (0,255,0), 3)

In [98]:
cv2.imwrite("6paper.jpg", paper)
cv2.imwrite("7warped.jpg", warped)
warped=cv2.cvtColor(paper,cv2.COLOR_BGR2GRAY)

In [99]:
thresh = cv2.threshold(warped, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [100]:
thresh.shape

(600, 430)

In [101]:
cv2.imwrite("8thresh.jpg", thresh)
thresh=cv2.resize(thresh, (430,600))


In [102]:
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
	cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []
 
# loop over the contours
for c in cnts:
	# compute the bounding box of the contour, then use the
	# bounding box to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(c)
	ar = w / float(h)
	approx = cv2.approxPolyDP(c,0.01*cv2.arcLength(c,True),True)
	area = cv2.contourArea(c)
	# in order to label the contour as a question, region
	# should be sufficiently wide, sufficiently tall, and
	# have an aspect ratio approximately equal to 1
	if ((w >= 12 and h >=12) and (w<=16 and h<=16) and len(approx)>=8 and (area>90 and area<200) and ar >= 0.7 and ar <=1.5):
		questionCnts.append(c)

In [103]:
questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
correct = 0
minm = []
maxm = []
flag=False
count=-1;
ans_array=[]
len(questionCnts)

198

In [104]:
for (q, i) in enumerate(np.arange(0, len(questionCnts), 4)):
	count+=1
	# sort the contours for the current question from
	# left to right, then initialize the index of the
	# bubbled answer
	cnts = contours.sort_contours(questionCnts[i:i + 4])[0]
	bubbled = None
	flag=False
	# loop over the sorted contours
	for (j, c) in enumerate(cnts):
		# construct a mask that reveals only the current
		# "bubble" for the question
		mask = np.zeros(thresh.shape, dtype="uint8")
		cv2.drawContours(mask, [c], -1, 255, -1)

		# apply the mask to the thresholded image, then
		# count the number of non-zero pixels in the
		# bubble area
		mask = cv2.bitwise_and(thresh, thresh, mask=mask)
		total = cv2.countNonZero(mask)
		if (flag and total>135):
			bubbled=None
			break
		if(total>135):
			flag=True
            
		if(total>135):
			bubbled=(total,j)
          
		# if the current total has a larger number of total
		# non-zero pixels, then we are examining the currently
		# bubbled-in answer
		##if bubbled is None or total > bubbled[0]:
			##bubbled = (total, j) # What ???

	# initialize the contour color and the index of the
	# *correct* answer
	color = (0, 0, 255)
	k = ANSWER_KEY[q]
	
	# check to see if the bubbled answer is correct
	if bubbled and k == bubbled[1]:
		color = (0, 255, 0)
		correct += 1
		ans_array.append(k)       
	elif not bubbled:
		ans_array.append("NA")
	elif bubbled and k!=bubbled[1]:
		ans_array.append(bubbled[1])  
	# draw the outline of the correct answer on the test
	cv2.drawContours(paper, [cnts[k]], -1, color, 3)

KeyError: 18

In [83]:
questions=cv2.drawContours(warped.copy(), questionCnts, -1, (0,255,0), 3)
##change thickness from 3

In [84]:
cv2.imwrite("9questions.jpg", questions)

True

In [85]:
score = (correct / 18.0) * 100
print(correct)
print("[INFO] Score: {:.0f}%".format(score))
cv2.putText(paper, "{:.0f}%".format(score), (10, 30),
cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Result", paper)
cv2.imwrite("Result.jpg", paper)
cv2.waitKey(10000)
cv2.destroyAllWindows()

5
[INFO] Score: 28%


In [86]:
print(ans_array)

[1, 'NA', 'NA', 0, 'NA', 1, 'NA', 1, 'NA', 'NA', 1, 'NA', 1, 'NA', 'NA', 2, 'NA', 1]


In [ ]:
## def write_to_csv(ans_array, filename):
    file=open(filename, "w+")
    for x in ans_array:
        file.write(str(x))
        file.write(',')
    file.close()    

In [23]:
write_to_csv(ans_array, "abc.txt")